# Orality vs. Literacy in Dialogue

In this notebook, we will explore the relationship between patterns of oral language and patterns of written language. 

In [ ]:
import json
import nltk
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from scipy import sparse
from collections import Counter
import operator

Outlining Features of Spoken Language